#**5. User Engagement and Growth Strategy**

In [1]:
import sys
import os

sys.path.append(os.path.abspath("D:/D26_Files/Phonepe_Analytics/Pulse_Case_Studies/"))

from db_connection import get_phonepe_engine


engine = get_phonepe_engine()

In [2]:
import pandas as pd
from sqlalchemy import text

In [3]:
tables_df = pd.read_sql("SHOW TABLES;", engine)
tables_df

,Tables_in_phonepe_db
0,agg_transaction
1,agg_user
2,insurance_transaction
3,map_transaction
4,map_user
5,top_transaction


In [4]:
Agg_User_df = pd.read_sql("SELECT * FROM agg_user;", engine)
Agg_User_df.head()

,State,Year,Quarter,Registered_users,App_opens,Brand,Brand_count,Brand_percentage
0,Andaman & Nicobar Islands,2018,1,6740,0,Xiaomi,1665,0.247033
1,Andaman & Nicobar Islands,2018,1,6740,0,Samsung,1445,0.214392
2,Andaman & Nicobar Islands,2018,1,6740,0,Vivo,982,0.145697
3,Andaman & Nicobar Islands,2018,1,6740,0,Oppo,501,0.074332
4,Andaman & Nicobar Islands,2018,1,6740,0,OnePlus,332,0.049258


In [5]:
Map_User_df = pd.read_sql("SELECT * FROM map_user;", engine)
Map_User_df.head()

,State,Year,Quarter,District,Registered_users,App_opens
0,Andaman & Nicobar Islands,2018,1,North And Middle Andaman District,632,0
1,Andaman & Nicobar Islands,2018,1,South Andaman District,5846,0
2,Andaman & Nicobar Islands,2018,1,Nicobars District,262,0
3,Andaman & Nicobar Islands,2018,2,North And Middle Andaman District,911,0
4,Andaman & Nicobar Islands,2018,2,South Andaman District,8143,0


In [6]:
Map_User_df['Year'].unique()

array([2018, 2019, 2020, 2021, 2022, 2023, 2024])

*1.	Which states/districts show the highest number of registered users?*

In [ ]:
top_states_df = pd.read_sql("""
    SELECT 
        State,
        SUM(Registered_users) as Total_Users
    FROM agg_user
    WHERE Year = (SELECT MAX(Year) FROM agg_user) 
    GROUP BY State 
    ORDER BY Total_Users DESC
    LIMIT 10
""", engine)
top_states_df

In [ ]:
top_districts_df = pd.read_sql("""
    SELECT 
        State,
        District,
        SUM(Registered_users) as Total_Users
    FROM map_user
    WHERE Year = (SELECT MAX(Year) FROM map_user)  
    GROUP BY State, District 
    ORDER BY Total_Users DESC
    LIMIT 10
""", engine)
top_districts_df

*2.	How does the ratio of active users (app opens) to registered users vary by top 10 states/regions*

In [ ]:
state_ratio_df = pd.read_sql("""
    SELECT 
        State,
        SUM(Registered_users) as Total_Registered,
        SUM(App_opens) as Total_App_Opens,
        ROUND((SUM(App_opens) * 1.0 / SUM(Registered_users)) * 100, 2) as Engagement_Ratio_Percent
    FROM agg_user 
    WHERE Registered_users > 0 AND Year = (SELECT MAX(Year) FROM agg_user)
    GROUP BY State 
    ORDER BY Engagement_Ratio_Percent DESC
    LIMIT 10;
""", engine)
state_ratio_df

In [ ]:
district_ratio_df = pd.read_sql("""
    SELECT 
        State,
        District,
        SUM(Registered_users) as Total_Registered,
        SUM(App_opens) as Total_App_Opens,
        ROUND((SUM(App_opens) * 1.0 / SUM(Registered_users)) * 100, 2) as Engagement_Ratio_Percent
    FROM map_user 
    WHERE Registered_users > 0 AND Year = (SELECT MAX(Year) FROM map_user)
    GROUP BY State, District 
    ORDER BY Engagement_Ratio_Percent DESC
    LIMIT 10;
""", engine)
district_ratio_df

*3.	Which 10 regions show high registrations but poor engagement (dormant users)?*

In [ ]:
dormant_user_df = pd.read_sql("""
    SELECT 
        State,
        SUM(Registered_users) as Total_Registered,
        SUM(App_opens) as Total_App_Opens,
        ROUND((SUM(App_opens) * 1.0 / SUM(Registered_users)) * 100, 2) as Engagement_Ratio_Percent,'Dormant Region' as Category
    FROM agg_user 
    WHERE Registered_users > 0 AND Year = (SELECT MAX(Year) FROM agg_user)
    GROUP BY State 
    ORDER BY Total_Registered DESC, Engagement_Ratio_Percent ASC
    LIMIT 10
""", engine)
dormant_user_df

*4.	What are the growth of states in terms of user engagement over time?*

In [ ]:
yearly_growth_df = pd.read_sql("""
    SELECT 
        State,
        Year,
        ROUND(
            (SUM(App_opens) * 1.0 / SUM(Registered_users)) * 100, 2
        ) as Yearly_Engagement_Percent
    FROM agg_user 
    GROUP BY State, Year 
    ORDER BY Yearly_Engagement_Percent DESC
""", engine)
yearly_growth_df

*5.	Which 20 districts could be targeted to increase user stickiness?*

In [ ]:
target_districts_df = pd.read_sql("""
    SELECT
    State,
    District,
    SUM(Registered_users) AS Total_Registered,
    SUM(App_opens) AS Total_App_Opens,
    ROUND((SUM(App_opens) * 1.0 / NULLIF(SUM(Registered_users), 0)) * 100, 2) AS Engagement_Ratio_Percent
FROM map_user
WHERE Year = (SELECT MAX(Year) FROM map_user) 
GROUP BY State, District
HAVING Engagement_Ratio_Percent > 0
ORDER BY Engagement_Ratio_Percent ASC
LIMIT 20;
""", engine)
target_districts_df